In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/2.url'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

from urllib.parse import urlparse

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

list_series_error, df_input = [], pd.read_excel('./file/1.page.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_series_error, count_remain

    (crawler_index, crawler_series) = crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        list_url = []
        for count_cycle in range(5):

            for page in range(crawler_series['Page']):

                count_retry = 0
                while True:
                    count_retry += 1

                    try:
                        url_request = f'''{crawler_series['Url']}&page={page}'''

                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            list_href = html.xpath('//div[@class="catalog-product marketplace-product row"]/div[@class="product-details-col col-md-10 row"]/div[@class="col-xs-12 col-md-8"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href') + html.xpath('//div[@class="catalog-product row"]/div[@class="product-details-col col-xs-7"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href')
                            if list_href:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_url.extend([f'https://{urlparse(url_request).netloc}{href.split('?')[0].strip()}' for href in list_href])
                list_url = list(dict.fromkeys(list_url))

                # = = = = = = = = = = = = = = =

                if count_remain % 3 == 0 and count_remain != 0:
                    clear_output()
                print(f'''[第{count_cycle + 1}轮] - [第{page + 1}页] - [请求{count_retry}次] - [{len(list_url)}/{crawler_series['SKU Count']}] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

                # = = = = = = = = = = = = = = =

                if len(list_url) == crawler_series['SKU Count']:
                    break

        # = = = = = = = = = = = = = = =

        if not list_url:
            raise

        # = = = = = = = = = = = = = = =

        df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                                   'Url': list_url})
        df_correct.to_excel(f'''{directory_out}/crawler_{crawler_series['No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/url_error.xlsx', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[第1轮] - [第1页] - [请求1次] - [18/1376] - [剩余1条] - [2024/11/11 15:08:57] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=0

[第1轮] - [第1页] - [请求1次] - [18/1183] - [剩余1条] - [2024/11/11 15:08:57] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&page=0

[第1轮] - [第2页] - [请求1次] - [18/1183] - [剩余1条] - [2024/11/11 15:09:00] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&page=1

[第1轮] - [第2页] - [请求1次] - [18/1376] - [剩余1条] - [2024/11/11 15:09:00] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=1

[第1轮] - [第3页] - [请求1次] - [36/1183] - [剩余1条] - [2024/11/11 15:09:03] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&page=2

[第1轮] - [第3页] - [请求1次] - [36/1376] - [剩余1条] - [2024/11/11 15:09:04] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator&page=2

[第1轮] - [第4页] - [请求1次] - [54/1183] - [剩余1条] - [2024/11/11 15:09:06] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator&

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 1027.26it/s]

Done ~



C:\Users\Administrator\Desktop\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
